In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pyodbc
import mysql.connector
import json
import matplotlib.pyplot as plt
import seaborn as sns
from decimal import Decimal

import requests
import datetime
from tqdm import tqdm

In [2]:
pyodbc.drivers()

['MySQL']

In [3]:
# Redshift 연결 정보
host='dev-data-etl-rs.ct0xxcn9xq5g.ap-northeast-2.redshift.amazonaws.com'
dbname='data_analysis_db'
user='stella'
password='iWntan282E5jJQc'
port='5439'

In [4]:
# Redshift에 연결
conn = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password,
    port=port
)

In [5]:
# 커서 생성
cursor = conn.cursor()
# SQL 쿼리 실행 (예시: frigga_sample 테이블에서 모든 데이터를 조회)
cursor.execute("""                          
                    SELECT 
                        *
                    FROM 
                        "data_analysis_db"."da_schema"."t_frigga_weather_map_alert"
                    WHERE 1=1
                        -- AND device_serial='VC6M6094'
                    ORDER BY device_datetime_kor ASC
                ;
               """)

# 결과 가져오기
rows = cursor.fetchall()

cursor.close()
conn.close()

In [6]:
# 컬럼명 리스트 정의
columns = ['device_datetime_kor', 'device_serial', 'lat', 'lon', 'battery'
           , 'temperature', 'humidity', 'acc_orig', 'acc', 'accx', 'accy', 'accz'
           , 'tiltx', 'tilty', 'tiltz', 'measure_date_kst', 'aws_station_id'
           , 'aws_station_name', 'aws_lat', 'aws_lon', 'distance_km', 'ws1', 'ws10', 'ta', 'hm'
           , 'min_temp', 'max_temp', 'mean_temp', 'std_temp'
           , 'min_humi', 'max_humi', 'mean_humi', 'std_humi'
           , 'min_acc', 'max_acc', 'mean_acc', 'std_acc'
           , 'Q1_temp', 'Q3_temp', 'temp_start', 'temp_end'
           , 'Q1_humi', 'Q3_humi', 'humi_start', 'humi_end'
           , 'Q1_acc', 'Q3_acc', 'acc_start', 'acc_end'
           , 'alert_temp', 'alert_humi', 'alert_acc', 'item_shock']

# rows를 DataFrame으로 변환
df= pd.DataFrame(rows, columns=columns)

# 데이터프레임 확인
print("데이터프레임 크기:", df.shape)
print("\n처음 5행 데이터:")
print(df.head())

데이터프레임 크기: (146528, 53)

처음 5행 데이터:
  device_datetime_kor device_serial        lat         lon battery  \
0 2024-11-14 18:57:04      VC6M60CA  37.420428  126.572040   100.0   
1 2024-11-14 19:07:04      VC6M60CA  37.420428  126.572040   100.0   
2 2024-11-14 19:17:04      VC6M60CA  37.420428  126.572040   100.0   
3 2024-11-14 19:27:04      VC6M60CA  37.420428  126.572040   100.0   
4 2024-11-14 19:37:04      VC6M60CA  37.420428  126.572040   100.0   

  temperature humidity acc_orig  acc accx  ... humi_start humi_end Q1_acc  \
0        18.7     58.0      1.0  1.0  0.0  ...      53.50    65.50   0.91   
1        18.5     59.0      0.9  0.9  0.0  ...      53.50    65.50   0.91   
2        18.7     58.0      1.0  1.0  0.0  ...      53.50    65.50   0.91   
3        18.6     59.0      0.9  0.9  0.0  ...      53.50    65.50   0.91   
4        18.5     59.0      0.9  0.9  0.0  ...      53.50    65.50   0.91   

  Q3_acc acc_start acc_end  alert_temp alert_humi alert_acc item_shock  
0   0.9

In [15]:
df.groupby('item_shock').size().count

<bound method Series.count of item_shock
정상    146440
파손        88
dtype: int64>

In [16]:
df.groupby(['device_serial', 'item_shock']).size().reset_index(name='count')

,device_serial,item_shock,count
0,VC6M6002,정상,925
1,VC6M6003,정상,829
2,VC6M6003,파손,1
3,VC6M6007,정상,3916
4,VC6M6007,파손,3
...,...,...,...
204,VC6M61C4,정상,12
205,VC6M61ED,정상,10
206,VC6ML002,정상,402
207,VC6ML002,파손,7
